## Setup

In [5]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix
import mlflow

from src.config import RAW_DATA_PATH, RANDOM_SEED
from src.feature_engineering import ChurnFeatureEngineer
from src.train import ChurnModelTrainer

## Quick model comparison

In [6]:
# Load and prepare data
df = pd.read_csv(RAW_DATA_PATH / "WA_Fn-UseC_-Telco-Customer-Churn.csv")
trainer = ChurnModelTrainer()
X_train, X_val, X_test, y_train, y_val, y_test, encoder = trainer.prepare_data(df)

# Train baseline
baseline_model, scaler, baseline_auc = trainer.train_baseline(X_train, X_val, y_train, y_val)

# Train XGBoost
xgb_model, xgb_auc = trainer.train_xgboost(X_train, X_val, y_train, y_val)

print(f"Baseline AUC: {baseline_auc:.4f}")
print(f"XGBoost AUC: {xgb_auc:.4f}")
print(f"Improvement: {((xgb_auc - baseline_auc) / baseline_auc * 100):.1f}%")


INFO:src.train:Preparing data for training...
INFO:src.feature_engineering:Created 31 features
INFO:src.train:Train shape: (4225, 30), Val shape: (1409, 30), Test shape: (1409, 30)
INFO:src.train:Class distribution - Train: 26.53%, Val: 26.54%, Test: 26.54%
INFO:src.train:Training baseline model...
2025/08/05 17:38:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/anaconda3/envs/ml-dev/lib/python3.9/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing v

Baseline AUC: 0.8399
XGBoost AUC: 0.8321
Improvement: -0.9%
